<a href="https://colab.research.google.com/github/paulodowd/EMATM0053_21_22/blob/main/WLabsheets/L4_ObstacleAvoidance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labsheet 4: Obstacle Avoidance

In previous labsheets we have investigated:
- making the robot move.
- bang-bang control.
- weighted-sensing for line following.
- proportional control.
- Finite State Machines, transitioning between states or robot behaviours.

In this labsheet, we will investigate the new required behaviour Obstacle Avoidance.  



<hr><br><br><br><br>

## E-Puck Proximity Sensors

Within the provided Webots world file, your simulated robot should meet an obstruction along the line following map.  This obstruction is a pink block.  The exact location of the pink block is not critical, however it should be investigated by you for the challenge is presents:

<p align="center">
<img src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_Obstruction.png?raw=true">
</p>

Your simulated robot is required to detect this block and safely navigate around it, to rejoin the line and continue line following.  To achieve this functionality, we will use the proximity sensors of the e-puck robot.

The simulated e-puck robot has 8 infra-red proximity sensors, positioned around the robot body.  These sensors are enumerated (indexed) following the same scheme in the below diagram (`ps0` ... `ps7`, "proximity sensor"):

<p align="center">
<img src="https://raw.githubusercontent.com/cyberbotics/webots/released/docs/guide/images/robots/epuck/sensors_and_leds.png">
</p>

Similar to the ground sensor, the proximity sensor transmits a beam of infra-red light and measures the intensity of the reflection.  These sensors have been modelled in Webots to have the following **`sensor response`**:

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_EpuckProxResponse.png?raw=true">
</p>

In the above plot from the <a href="https://cyberbotics.com/doc/guide/epuck#e-puck-model">Webots documentation</a>, we can observe:
- the sensor response is `non-linear`.
- the sensor response is `high` at `close proximity`.
- the sensor response is `low` at `far proximity`.
- the maximum range of the sensor is 7cm.
- the measured signal flattens out at a value of approximately `34`, an offset bias.
- there is some noise in the measurement signal.

## Exercise 1: E-Puck Model

1. If the maximum raw measurement from the `analog-to-digital` converter (ADC) is 4095, how many bits of resolution must the ADC be configured to?
  - why might this be a different resolution to the ground sensor?

2. Would you regard this sensor as `precise`?
  - when is this sensor `accurate`?

3. The shape of the `sensor response` is `non-linear`, and this has implications for the robot's ability to perceive the environment:
  - measurements below the approximate value of 306 on the y-axis suffer from **`perceptual aliaising`** - what does this mean?

4. How could you normalise a sensor reading measurement to be within the range [ 0.0 : 1.0 ]?

5. What steps might you take to convert the y-axis values into units of distance?
  - in what scenario or task would this unit conversion be useful?

6. What characteristic of the `sensor response` could be used to give a measure of confidence to a given sensor reading?
  - within the context of obstacle avoidance, what does the term `confidence` refer to?





<hr><br><br><br><br>

## Obstacle Avoidance Behaviour

When developing Obstacle Avoidance behaviour, it is recommended to put aside any previous line following behaviour.  It is better to simplify the problem first.  Once obstacle avoidance is achieved, the complexity of the problem can be increased to include line following behaviour.  

<p align="center">
<img width="50%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_ObsAvoid.png?raw=true">
</p>

For obstacle avoidance behaviour, it is necessary for the robot to decide:
- is there an obstruction?
  - a logical choice, similar to the bang-bang controller operation.
- how to utilise the 8 sensors?
  - a more complex issue, where a form of weighted-measurement might be useful.

In Labsheet 2 we implemented a `weighted-measurement` for line following.  This was expressed as the equations:

<p align="center">
$w_{left} = ( gs_{0} + ( gs_{1} * 0.5 ) )$
</p>
<p align="center">
$w_{right} = ( gs_{2} + ( gs_{1} * 0.5 ) )$
</p>

<p align="center">
$e_{line} = w_{left} - w_{right}$
</p>

where `gs0`, `gs1`, and `gs2` correspond to the left, centre and right sensing elements of the ground sensor.  In the above equations, we can see that the value of `gs1` is weighted by 0.5.  

As a matter of interest, we can also reconceptualise this technique with an illustration of the form of a very simple <a href="https://en.wikipedia.org/wiki/Artificial_neural_network">artificial neural network</a>:

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_LineANN.png?raw=true">
</p>

In the above illustration, the value from each node is fed forwards, as indicated by the arrows.  So in the first row of nodes ("input layer"), the nodes labelled `gs0`, `gs1`, `gs21` receive the latest value of the ground sensors.  The small numbers next to the arrows represent `weights`.  When a weight is `1.0`, the value passed forwards is unmodified.  Each node in the network operates a simple summation of the inputs received.

This representation of weighting inputs to produce a useful output is particularly useful for designing obstacle avoidance.  In the case of the e-puck robot, there are 8 infra-red proximity sensors, and we must decide how they can be used to effect obstacle avoidance behaviour. 

<p align="center">
<img width="75%" src="https://github.com/paulodowd/EMATM0053_21_22/blob/main/images/Webots_ObsANN.png?raw=true">
</p>

In the above illustration, it is necessary to decide which sensors (`ps0` to `ps7`) will be fed forwards to the output node, $e_{obs}$ (error - obstruction), and with what weighting.  Here, we are assuming a similar principle of operation, where $e_{obs}$ would be an error signal within a positive and negative range.

<p align="center">
<img src="https://raw.githubusercontent.com/cyberbotics/webots/released/docs/guide/images/robots/epuck/sensors_and_leds.png">
</p>

If we consider the above body-plan of the e-puck robot, we can see that `ps7` and `ps0` face directly forwards.  We can imagine that encountering an obstruction on these sensors would cause a rapid turn.  Furthermore, for `ps7`, the robot might need to turn right, and for `ps0` the robot might need to turn left.  


Within Webots, we can access the e-puck proximity sensors with the following example code extract:

```c

  // Loop through all proximity sensors [0:7]
  int i;
  for( i = 0; i < NB_PS; i++ ) {

    // read value from sensor
    ps_value[i] =  wb_distance_sensor_get_value(ps[i]);
    
    // Print this value to the console to inspect.
    printf("%f,", ps_value[i]);
  }
  printf("\n");
```

The following code extract illustrates how `weights` could be used to generate an output signal, $e_{obs}$:

```c
int i;
// Same number of weigths as sensors
float weights[NB_PS] = { 0.1, 0.2, 0.4, 0.5, -0.5, -0.4, -0.2, -0.1 };
float e_obs;

// Set initial value.
e_obs = 0.0;
for( i = 0; i < NB_PS; i++ ) {

    // read value from sensor
    ps_value[i] =  wb_distance_sensor_get_value(ps[i]);

    // Simple summation of weighted sensor values.
    e_obs = e_obs + ( ps_value[i] * weights[i] );

}

```

After the above example code, you might utilise $e_{obs}$ in a manner similar to the Line Following exercises.  Here, it is suggested that you use $e_{obs}$, but it is also possible to design a very simple ANN to utilise two output nodes, one to control the left motor speed and one to control the right motor speed. 

## Exercise 2: Obstacle Avoidance

1. Before you start writing code consider:
  - how could obstacle avoidance behaviour be achieved with a bang-bang controller?
  - what advantages or disadvantages might there to a bang-bang controller?
 
2. Decide a threshold value from the `sensor response curve` for when an `obstacle avoidance behaviour` should be activated.  
  - **Note:** the proximity sensors always report a value, even when there is not an obstruction. 
  - write a function to check all proximity sensors and return an appropriate value if obstacle avoidance is required.
  - check that this function operates reliably.
  - utilise the result of this function to either:
    - transition your FSM into/out-of obstacle avoidance appropriately (preferred)
    - activate obstacle avoidance execution flow in your program

3. Start by creating a weight of 0 for all sensors.  With just the two foward proximity sensors, find weight values to effect a turning behaviour in the approriate direction:
  - which two proximity senors are the most forward facing?
  - how might you pre-process your proximity sensor measurements to make subsequent calculations easier?
  - **help:** to begin with, do not have your robot move forward or backward, only turn on the spot.  
  - **help:** remember that you can position your robot within the environment by clicking on it.
  - **help:** avoid **`blocking code`** - for exmaple, your obstacle avoidance function should return and be called again by `loop()` even while there is an obstruction.
  - what would be a desirable value of `e_obs` if both forward sensors report identical values, or cancel each other out?  Implement a bias in your gain values to solve this issue.  
    - **Hypothesise:** in what environment circumstances would the robot be in where both sensors report the same value?
  - **Validate:** using just the two forward sensors, how far does your robot rotate away from the obstruction?  Would this enough to transit without collision?
  - **help**: the following code extract follows a similar format to line following:


  ```c

void loop() {

  // Get the line error
  float e_obs;
  e_obs = getObstacleError();

  // Determine a proportional rotation speed
  float turn_velocity;
  turn_velocity = ????;  // What is a sensible maximum speed?
  turn_velocty = turn_velocity * e_obs;

  // Set motor values.
  // What does "0 -" and "0 +" achieve here?
  wb_motor_set_velocity(right_motor, 0 - turn_velocity);
  wb_motor_set_velocity(left_motor, 0 + turn_velocity);
  
}

// A function to return an error signal representative
// of the line placement under the ground sensor.
float getObstacleError() {
  float e_obs;

  // Read proximity sensors, store result

  // Calculated error signal
  e_obs = ????;

  // Return result
  return e_obs;
}

```

3. Progressively implement the remaining proximity sensors into your obstacle avoidance function:
  - are all sensors necessary?

4. Implement a forward velocity for your obstacle avoidance:
  - consider if this can be `proportionally controlled`, and whether this is useful.  What would be a useful measurement for forward velocity control?
  - is a backward velocity useful?

5. How might you calculate a measure of obstacle avoidance performance?
  - what `proprioceptive` information is available to the robot?
  - what `exteroceptive` information is available to the robot?
  - what features of the robot behaviour are more or less desirable, and how could these be represented mathematically within this `metric` of performance?
  - how could this `metric` be defined and calculated to make it as transferable (comparable) as possible between different robotic systems?
  - **help**: in machine learning, we might consider this the <a href="https://en.wikipedia.org/wiki/Reinforcement_learning">reward function</a> or the <a href="https://en.wikipedia.org/wiki/Evolutionary_algorithm">fitness function</a>.  
  



<hr><br><br><br><br>